In [ ]:
from IPython.core.display import display, HTML
display(HTML('<style>.container {width: 90% !important; }</style>'))

In [ ]:
import sys
sys.path.append("../../simpsom")
import simpsom as sps

import pandas as pd
import numpy as np

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
df = pd.DataFrame(digits.data, columns=digits.feature_names)
labels = digits.target
data = np.array(df)

In [ ]:
def run_experiment(data, PBC=True, init='PCA', metric='euclidean', topology='hexagonal', 
               GPU=False, neighborhood_fun='gaussian', train_algo='batch', early_stop=None,
               plotall=False, size=10, epochs=10):

    net = sps.SOMNet(size, size, data, topology=topology, PBC=PBC, init=init, metric=metric, random_seed=32, GPU=GPU, debug=True)
    net.train(train_algo=train_algo, start_learning_rate=0.01, epochs=epochs, batch_size=-1, early_stop=early_stop)

    if plotall:

        net.plot_map_by_feature(feature=1, show=True, print_out=False)
        _ = net.plot_map_by_difference(show=True, print_out=False, returns=False)

        _ = net.plot_projected_points(net.project_onto_map(data), 
                                     project=False, jitter=True, 
                                     show=True, print_out=False)
        _ = net.plot_projected_points(data, color_val=[n.difference for n in net.nodes_list],
                                     project=True, jitter=True, 
                                     show=True, print_out=False)

        labs, points = net.cluster(data, algorithm='AgglomerativeClustering')
        net.plot_clusters(data, labs, project=True, show=True)
        labs, points = net.cluster(data, algorithm='DBSCAN')
        net.plot_clusters(data, labs, project=True, show=True)
        labs, points = net.cluster(data, algorithm='KMeans')
        net.plot_clusters(data, labs, project=True, show=True)

        net.plot_convergence(fsize=(5, 5), logax=False)

In [ ]:
GPU=False

confs = [
{'PBC': False, 'init':'PCA', 'metric':'euclidean', 'topology':'hexagonal', 'GPU':GPU, 'neighborhood_fun':'gaussian', 'train_algo':'online', 'early_stop':None, 'plotall':False},
{'PBC': False, 'init':'PCA', 'metric':'cosine', 'topology':'hexagonal', 'GPU':GPU, 'neighborhood_fun':'gaussian', 'train_algo':'online', 'early_stop':None, 'plotall':False},
{'PBC': False, 'init':'PCA', 'metric':'manhattan', 'topology':'hexagonal', 'GPU':GPU, 'neighborhood_fun':'gaussian', 'train_algo':'online', 'early_stop':None, 'plotall':False},
{'PBC': False, 'init':'random', 'metric':'euclidean', 'topology':'hexagonal', 'GPU':GPU, 'neighborhood_fun':'gaussian', 'train_algo':'online', 'early_stop':None, 'plotall':False},
{'PBC': False, 'init':(np.array([[0,1]]*data.shape[1])).T, 'metric':'euclidean', 'topology':'hexagonal', 'GPU':GPU, 'neighborhood_fun':'gaussian', 'train_algo':'online', 'early_stop':None, 'plotall':False},
{'PBC': True, 'init':'PCA', 'metric':'euclidean', 'topology':'hexagonal', 'GPU':GPU, 'neighborhood_fun':'gaussian', 'train_algo':'online', 'early_stop':None, 'plotall':False},
{'PBC': False, 'init':'PCA', 'metric':'euclidean', 'topology':'square', 'GPU':GPU, 'neighborhood_fun':'gaussian', 'train_algo':'online', 'early_stop':None, 'plotall':False},
{'PBC': False, 'init':'PCA', 'metric':'euclidean', 'topology':'hexagonal', 'GPU':GPU, 'neighborhood_fun':'mexican', 'train_algo':'online', 'early_stop':None, 'plotall':False},
{'PBC': False, 'init':'PCA', 'metric':'euclidean', 'topology':'hexagonal', 'GPU':GPU, 'neighborhood_fun':'bubble', 'train_algo':'online', 'early_stop':None, 'plotall':False},
{'PBC': False, 'init':'PCA', 'metric':'euclidean', 'topology':'hexagonal', 'GPU':GPU, 'neighborhood_fun':'gaussian', 'train_algo':'batch', 'early_stop':None, 'plotall':False},
{'PBC': True, 'init':'PCA', 'metric':'euclidean', 'topology':'square', 'GPU':GPU, 'neighborhood_fun':'gaussian', 'train_algo':'online', 'early_stop':None, 'plotall':False},
{'PBC': True, 'init':'PCA', 'metric':'euclidean', 'topology':'hexagonal', 'GPU':GPU, 'neighborhood_fun':'gaussian', 'train_algo':'batch', 'early_stop':None, 'plotall':False},
{'PBC': False, 'init':'PCA', 'metric':'cosine', 'topology':'hexagonal', 'GPU':GPU, 'neighborhood_fun':'gaussian', 'train_algo':'batch', 'early_stop':None, 'plotall':False},
{'PBC': False, 'init':'PCA', 'metric':'manhattan', 'topology':'hexagonal', 'GPU':GPU, 'neighborhood_fun':'gaussian', 'train_algo':'batch', 'early_stop':None, 'plotall':False},
{'PBC': False, 'init':'PCA', 'metric':'euclidean', 'topology':'hexagonal', 'GPU':GPU, 'neighborhood_fun':'gaussian', 'train_algo':'online', 'early_stop':'bmudiff', 'plotall':False},
{'PBC': False, 'init':'PCA', 'metric':'euclidean', 'topology':'hexagonal', 'GPU':GPU, 'neighborhood_fun':'gaussian', 'train_algo':'online', 'early_stop':'mapdiff', 'plotall':False}, 
{'PBC': True, 'init':'PCA', 'metric':'euclidean', 'topology':'square', 'GPU':GPU, 'neighborhood_fun':'gaussian', 'train_algo':'online', 'early_stop':None, 'plotall':True},
{'PBC': False, 'init':'PCA', 'metric':'euclidean', 'topology':'hexagonal', 'GPU':GPU, 'neighborhood_fun':'gaussian', 'train_algo':'batch', 'early_stop':'mapdiff', 'plotall':True},
{'PBC': False, 'init':'PCA', 'metric':'euclidean', 'topology':'square', 'GPU':GPU, 'neighborhood_fun':'gaussian', 'train_algo':'online', 'early_stop':None, 'plotall':True},
]

In [ ]:
print('CPU')
for conf in confs:
    print(conf)
    run_experiment(data, **conf)
print('All CPU tests done!')

In [ ]:
try:
    import cupy
    print('GPU')
    for conf in confs:
        print(conf)
        conf['GPU']=True
        run_experiment(data, **conf)
    print('All GPU tests done!')
except ImportError:
    print('No CuPy found, GPU tests will be skipped')